In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.metrics import f1_score

In [2]:


def get_data(type="train"):
    data = []
    label = []
    for i in range(1, 37):
        X = []
        y = []
        target = f"{i:02d}"  
        train_data_path = f"data/q2/{type}/{target}"
        
        for img_name in os.listdir(train_data_path):
            img_path = os.path.join(train_data_path, img_name)
            
            
            img = cv2.imread(img_path)
            
            
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            
            img = cv2.resize(img, (32, 32))
            img = img / 255.0 
            img_flatten = img.flatten()
            X.append(img_flatten)
            y.append(i)

        data.extend(X)
        label.extend(y)

    data = np.array(data)
    label = np.array(label)
    return data, label

X_train , y_train = get_data("train")
X_test , y_test = get_data("test")
y_train  = np.eye(36)[y_train - 1].astype(np.float32)
y_test  = np.eye(36)[y_test - 1].astype(np.float32)


In [3]:


class Layer:
    def __init__(self,in_dimension , out_dimension , learning_rate = 0.001 , is_sigmoid = True,is_output = False):
        self.net_grad = None
        self.lr = learning_rate
        self.O = None
        limit = np.sqrt(6 / (in_dimension + out_dimension))
        self.w = np.random.uniform(-limit, limit, (in_dimension + 1, out_dimension)).astype(np.float32)

        self.output_function = self.__sigmoid if is_sigmoid else self.__relu
        self.output_function_grad = self.__sigmoid_grad if is_sigmoid else self.__relu_grad
        self.is_output = is_output

    def __relu(self,x):
        return np.maximum(0,x)

    def __relu_grad(self,x):
        return np.where(x > 0 , 1 , 0)

    def __sigmoid(self,x):
        return 1 / (1 + np.exp(-x))

    def __softmax(self,x):
        exp_x = np.exp(x - np.max(x , axis =1 , keepdims=True))
        return exp_x / np.sum(exp_x, axis = 1 , keepdims=True)

    def __sigmoid_grad(self,x):
        return x * (1-x)    

    def next(self,X ):
        X = np.hstack([np.ones((X.shape[0] , 1)),X])
        self.net_grad = np.dot(X, self.w)
        self.O = self.output_function(self.net_grad)
        return self.O

    def _prev(self, dO , O_down):
        if not self.is_output:
            new_grad = dO * self.output_function_grad(self.O)
            dw = np.dot(np.hstack([np.ones((O_down.shape[0], 1)), O_down]).T , new_grad) / O_down.shape[0]
            d_O_down = np.dot(new_grad , self.w.T)
            dO_prev =   d_O_down[:,1:]
            self.w = self.w - self.lr * dw
            return  dO_prev
        
            
    

class FinalLayer :
    def __init__(self,in_dim , out_dim , lr = 0.001 ,is_softmax = True) :
        self.lr = lr
        limit = np.sqrt(6 / (in_dim + out_dim))
        self.w = np.random.uniform(-limit, limit, (in_dim + 1, out_dim)).astype(np.float32)
        self.net_grad = None
        self.O = None
        self.output_function = self.__softmax if is_softmax else self.__sigmoid

    
           

    def __sigmoid(self,x):
        return 1 / (1 + np.exp(-x))

    def __softmax(self,x):
        exp_x = np.exp(x - np.max(x , axis =1 , keepdims=True))
        return exp_x / np.sum(exp_x, axis = 1 , keepdims=True)

    def __sigmoid_grad(self,x):
        return x * (1-x)  

    def next(self , X) :
        X = np.hstack([np.ones((X.shape[0] , 1)),X ])    
        self.net_grad = np.dot(X, self.w)
        self.O = self.output_function(self.net_grad)
        return self.O
    
    def _prev(self, y_true , O_down)  :
        new_grad = self.O - y_true
        dw = np.dot(np.hstack([np.ones((O_down.shape[0], 1)), O_down]).T , new_grad) / y_true.shape[0]
        d_O_down = np.dot(new_grad , self.w.T)
        dO_prev =   d_O_down[:,1:]
        self.w = self.w - self.lr * dw
        return dO_prev
    
class NeuralNetwork:
    def __init__(self, M = 32 , n = 3072 , HiddenLayer = [512 , 265 ]  , target_class = 36 , lr = 0.01,is_sigmoid = True):
        self.lr = lr
        self.layers = []
        self.Batch_size = M
        prev_dimn = n
        for h in HiddenLayer:
            layer = Layer(prev_dimn , h , is_sigmoid = is_sigmoid , learning_rate = self.lr)
            self.layers.append(layer)
            prev_dimn = h
        self.output_layer = FinalLayer(prev_dimn , target_class , is_softmax = True , lr = self.lr)

    def next(self,X) : 
        O_down = X 
        for layer in self.layers :
            O_down = layer.next(O_down)

        predictions = self.output_layer.next(O_down)
        return predictions
    
    def corss_entropy_loss(self, y_true , y_pred):
        m = y_true.shape[0]
        return - np.sum(y_true * np.log(y_pred + 1e-10 )) / m
     
    def _prev(self,Y , X) : 
        O = X 
        stored_outputs = [X]
        for layer in self.layers :
            O = layer.next(O)
            stored_outputs.append(O)
        predictions = self.output_layer.next(O)

       

        do_down = self.output_layer._prev(Y , stored_outputs[-1])
        for i in range(len(self.layers)-1 , -1 , -1):
            do_down = self.layers[i]._prev(do_down , stored_outputs[i])

        return predictions  

    def predict(self,X) :
        predictions = self.next(X)
        return np.argmax(predictions , axis = 1)   
    
    def fit(self , X , Y , epochs = 10 ):
        for epoch in range(epochs) : 
            comb = np.random.permutation(X.shape[0])
            X_shuffled = X[comb]
            Y_shuffled = Y[comb]
            J = 0

            for i in range(0, X.shape[0] , self.Batch_size):
                X_batch = X_shuffled[i : i + self.Batch_size]
                Y_batch = Y_shuffled[i : i + self.Batch_size]
                predictions = self._prev(Y_batch , X_batch)
                loss = self.corss_entropy_loss(Y_batch , predictions)
                J += loss
           
            print(f"Epoch {epoch + 1} / {epochs} , Loss : {J}")

               


In [4]:
hidden_layer_length = [[512],[512,256],[512,256,128],[512,256,128,64],]

for h_len in hidden_layer_length:
    model = NeuralNetwork(M = 32 , n = 3072 , HiddenLayer = h_len  , target_class = 36 , is_sigmoid = False , lr = 0.01) 
    model.fit(X_train , y_train , epochs = 20)
    y_test_prediction = model.predict(X_test)
    y_train_prediction = model.predict(X_train)
    print("Train Accuracy: ", np.mean(y_train_prediction == np.argmax(y_train , axis = 1)) * 100)
    print("Test Accuracy: ", np.mean(y_test_prediction == np.argmax(y_test , axis = 1)) * 100)
    
    def results_metrics(y_true, y_pred):
        f1 = f1_score(np.argmax(y_true , axis = 1) , y_pred , average='macro')
        precision = f1_score(np.argmax(y_true , axis = 1) , y_pred , average='macro', zero_division=0)
        recall = f1_score(np.argmax(y_true , axis = 1) , y_pred , average='macro', zero_division=0)
        return f1, precision, recall

    f1, precision, recall = results_metrics(y_test, y_test_prediction)
    print(f"f1 Score for hidden layer size {h_len} : ", f1)
    print(f"Precision for hidden layer size {h_len} : ", precision)
    print(f"Recall for hidden layer size {h_len} : ", recall)
    print("---------------------------------------------------")

    f1, precision, recall = results_metrics(y_train, y_train_prediction)
    print(f"Train f1 Score for hidden layer size {h_len} : ", f1)
    print(f"Train Precision for hidden layer size {h_len} : ", precision)
    print(f"Train Recall for hidden layer size {h_len} : ", recall)

Epoch 1 / 20 , Loss : 1410.3795818505146
Epoch 2 / 20 , Loss : 887.3334228372789
Epoch 3 / 20 , Loss : 766.7980125820359
Epoch 4 / 20 , Loss : 694.8222979784409
Epoch 5 / 20 , Loss : 635.6503266572336
Epoch 6 / 20 , Loss : 585.2483438553969
Epoch 7 / 20 , Loss : 539.1828790157534
Epoch 8 / 20 , Loss : 497.0867322085973
Epoch 9 / 20 , Loss : 458.83680171435776
Epoch 10 / 20 , Loss : 424.28964728745217
Epoch 11 / 20 , Loss : 393.043294739849
Epoch 12 / 20 , Loss : 364.4231414223733
Epoch 13 / 20 , Loss : 339.3621528425476
Epoch 14 / 20 , Loss : 315.3869720643986
Epoch 15 / 20 , Loss : 293.45137232770793
Epoch 16 / 20 , Loss : 274.6647893148358
Epoch 17 / 20 , Loss : 256.78075100842386
Epoch 18 / 20 , Loss : 240.91047587167682
Epoch 19 / 20 , Loss : 225.60601375560924
Epoch 20 / 20 , Loss : 211.7659058591298
Train Accuracy:  94.27777777777779
Test Accuracy:  84.93518518518518
f1 Score for hidden layer size [512] :  0.849187369703186
Precision for hidden layer size [512] :  0.8491873697031

In [5]:
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report
import numpy as np

hidden_layer_length = [
    [512],
    [512, 256],
    [512, 256, 128],
    [512, 256, 128, 64],
]

overall_results = []

for h_len in hidden_layer_length:
    print(f"\n========= Training Model with Hidden Layers: {h_len} =========")
    
    # Initialize model
    model = NeuralNetwork(M=32, n=3072, HiddenLayer=h_len, target_class=36, is_sigmoid=False, lr=0.01)
    
    # Train the model
    model.fit(X_train, y_train, epochs=20)
    
    # Predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Accuracy
    y_train_labels = np.argmax(y_train, axis=1)
    y_test_labels = np.argmax(y_test, axis=1)
    
    train_acc = np.mean(y_train_pred == y_train_labels) * 100
    test_acc = np.mean(y_test_pred == y_test_labels) * 100
    
    print(f"Train Accuracy: {train_acc:.2f}%")
    print(f"Test Accuracy: {test_acc:.2f}%")
    
    # Macro metrics
    def results_metrics(y_true_labels, y_pred):
        f1 = f1_score(y_true_labels, y_pred, average='macro', zero_division=0)
        precision = precision_score(y_true_labels, y_pred, average='macro', zero_division=0)
        recall = recall_score(y_true_labels, y_pred, average='macro', zero_division=0)
        return f1, precision, recall
    
    # Test metrics
    f1_test, precision_test, recall_test = results_metrics(y_test_labels, y_test_pred)
    print(f"\n📊 Test Macro Metrics:")
    print(f"F1 Score: {f1_test:.4f}")
    print(f"Precision: {precision_test:.4f}")
    print(f"Recall: {recall_test:.4f}")
    
    # Train metrics
    f1_train, precision_train, recall_train = results_metrics(y_train_labels, y_train_pred)
    print(f"\n📊 Train Macro Metrics:")
    print(f"F1 Score: {f1_train:.4f}")
    print(f"Precision: {precision_train:.4f}")
    print(f"Recall: {recall_train:.4f}")

    # ✅ Detailed per-class metrics for Test Data
    print("\n🔍 Detailed Per-Class Metrics (Test Data):")
    print(classification_report(y_test_labels, y_test_pred, digits=4, zero_division=0))

    # ✅ Detailed per-class metrics for Train Data
    print("\n🔍 Detailed Per-Class Metrics (Train Data):")
    print(classification_report(y_train_labels, y_train_pred, digits=4, zero_division=0))
    
    print("===================================================")
    
    # Append results for averaging
    overall_results.append({
        "layers": h_len,
        "train_acc": train_acc,
        "test_acc": test_acc,
        "f1_train": f1_train,
        "precision_train": precision_train,
        "recall_train": recall_train,
        "f1_test": f1_test,
        "precision_test": precision_test,
        "recall_test": recall_test
    })

# Compute average metrics across all models
avg_train_acc = np.mean([res["train_acc"] for res in overall_results])
avg_test_acc = np.mean([res["test_acc"] for res in overall_results])
avg_f1 = np.mean([res["f1_test"] for res in overall_results])
avg_precision = np.mean([res["precision_test"] for res in overall_results])
avg_recall = np.mean([res["recall_test"] for res in overall_results])

print("\n============== AVERAGE PERFORMANCE ACROSS ALL MODELS ==============")
print(f"Average Train Accuracy: {avg_train_acc:.2f}%")
print(f"Average Test Accuracy: {avg_test_acc:.2f}%")
print(f"Average Test F1 Score: {avg_f1:.4f}")
print(f"Average Test Precision: {avg_precision:.4f}")
print(f"Average Test Recall: {avg_recall:.4f}")
print("===================================================================")



========= Training Model with Hidden Layers: [512] =========
Epoch 1 / 20 , Loss : 1417.0166322760367
Epoch 2 / 20 , Loss : 880.0839094791822
Epoch 3 / 20 , Loss : 760.4211851574343
Epoch 4 / 20 , Loss : 687.5992048771793
Epoch 5 / 20 , Loss : 629.7357836509797
Epoch 6 / 20 , Loss : 578.6031068963745
Epoch 7 / 20 , Loss : 532.5367287154924
Epoch 8 / 20 , Loss : 490.17649100649487
Epoch 9 / 20 , Loss : 452.2570125952336
Epoch 10 / 20 , Loss : 417.2496015668978
Epoch 11 / 20 , Loss : 386.157857433306
Epoch 12 / 20 , Loss : 357.68938576596383
Epoch 13 / 20 , Loss : 332.50662490900936
Epoch 14 / 20 , Loss : 309.0680204723646
Epoch 15 / 20 , Loss : 287.720769218596
Epoch 16 / 20 , Loss : 269.31456291957306
Epoch 17 / 20 , Loss : 251.3456894027004
Epoch 18 / 20 , Loss : 235.03880321957462
Epoch 19 / 20 , Loss : 220.51272564859062
Epoch 20 / 20 , Loss : 207.0537379465492
Train Accuracy: 94.24%
Test Accuracy: 85.19%

📊 Test Macro Metrics:
F1 Score: 0.8517
Precision: 0.8548
Recall: 0.8519

📊 T